!!!!! The ID and element columns need to be removed to match the professor's example dataframe.

# DB access without download it 

In [7]:
from ftplib import FTP
import pandas as pd
from tqdm.notebook import tqdm
from io import BytesIO

# credential and host settings
ftp_host = 'download.mercatoelettrico.org'
ftp_username = 'FABIOINZOLI'
ftp_password = '12L16Z13'

# FTP connection
with FTP(ftp_host) as ftp:
    ftp.login(ftp_username, ftp_password)  # login

    # date range 
    start_date = pd.to_datetime('2023-01-01')
    end_date = pd.to_datetime('2023-12-31')
    date_range = pd.date_range(start=start_date, end=end_date)

    df_list = []  # Lista per memorizzare i DataFrame

    # iterate in range date 
    for date in tqdm(date_range, desc='Downloading files'):
        date_str = date.strftime('%Y%m%d')
        directory = '/MercatiElettrici/MGP_DomandaOfferta/'
        file_name = f'{date_str}MGPDomandaOfferta.xml'
        file_path = directory + file_name

        # change directory
        try:
            ftp.cwd(directory)
            if file_name in ftp.nlst():  # check
                with BytesIO() as bio:
                    ftp.retrbinary('RETR ' + file_name, bio.write)
                    bio.seek(0)  # return to the beginning of the file
                    df = pd.read_xml(bio)
                    df_list.append(df)
            else:
                print(f"{file_path} not found on FTP server")
        except Exception as e:
            print(f"Error accessing {file_path}: {e}")

        # return to the root directory
        ftp.cwd('/')

# concatenate all df in one 
ts_df = pd.concat(df_list, ignore_index=True)

print("Closing FTP connection")


Closing FTP connection


In [8]:
ts_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2965864 entries, 0 to 2965863
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            object 
 1   element       object 
 2   Data          float64
 3   Ora           float64
 4   Mercato       object 
 5   ZonaMercato   object 
 6   PrezzoZonale  float64
 7   Quantita      float64
 8   Prezzo        float64
 9   Tipo          object 
dtypes: float64(5), object(5)
memory usage: 226.3+ MB


In [9]:
ts_df.head()


,id,element,Data,Ora,Mercato,ZonaMercato,PrezzoZonale,Quantita,Prezzo,Tipo
0,NewDataSet,\n,NaN,NaN,None,None,NaN,NaN,NaN,None
1,None,None,20230101.0,1.0,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,195.9,70000.000,4000.0,OFF
2,None,None,20230101.0,1.0,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,195.9,0.102,3000.0,OFF
3,None,None,20230101.0,1.0,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,195.9,20.001,800.0,OFF
4,None,None,20230101.0,1.0,MGP,CALA;CNOR;CSUD;NORD;SARD;SICI;SUD;AUST;COAC;CO...,195.9,7.530,700.0,OFF


In [10]:
ts_df.Tipo.value_counts()

Tipo
OFF    1963855
BID    1001644
Name: count, dtype: int64

In [11]:
ts_df.to_pickle('ts_df.pkl.gz', compression='gzip')